In [2]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [10]:
data = {
    'Alice': [5, 3, 4, 4, np.nan],
    'User1': [3, 1, 2, 3, 3],
    'User2': [4, 3, 4, 3, 5],
    'User3': [3, 3, 1, 5, 4],
    'User4': [1, 5, 5, 2, 1]
}

item_names = [f'Item{i+1}' for i in range(len(data['Alice']))]

example_df = pd.DataFrame(data, index=item_names).T
print("Example DataFrame:")
display(example_df)

Example DataFrame:


,Item1,Item2,Item3,Item4,Item5
Alice,5.0,3.0,4.0,4.0,NaN
User1,3.0,1.0,2.0,3.0,3.0
User2,4.0,3.0,4.0,3.0,5.0
User3,3.0,3.0,1.0,5.0,4.0
User4,1.0,5.0,5.0,2.0,1.0


In [11]:
example_df_filled = example_df.fillna(0).T
print("\nExample DataFrame with NaN filled with 0:")
display(example_df_filled)
item_similarity_matrix = pd.DataFrame(cosine_similarity(example_df_filled),
                                      index=example_df_filled.index,
                                      columns=example_df_filled.index)
print("\nItem Similarity Matrix:")
display(item_similarity_matrix)


Example DataFrame with NaN filled with 0:


,Alice,User1,User2,User3,User4
Item1,5.0,3.0,4.0,3.0,1.0
Item2,3.0,1.0,3.0,3.0,5.0
Item3,4.0,2.0,4.0,1.0,5.0
Item4,4.0,3.0,3.0,5.0,2.0
Item5,0.0,3.0,5.0,4.0,1.0



Item Similarity Matrix:


,Item1,Item2,Item3,Item4,Item5
Item1,1.000000,0.780260,0.819782,0.943370,0.759257
Item2,0.780260,1.000000,0.942020,0.847984,0.673201
Item3,0.819782,0.942020,1.000000,0.784025,0.622425
Item4,0.943370,0.847984,0.784025,1.000000,0.811526
Item5,0.759257,0.673201,0.622425,0.811526,1.000000


In [12]:
def predict_ratings(user_id, item_id, user_item_matrix, item_similarity_matrix):
  user_ratings = user_item_matrix.loc[user_id].dropna()
  similar_items = item_similarity_matrix[item_id]
  sims_with_ratings = similar_items[user_ratings.index]
  positive_sims = sims_with_ratings[sims_with_ratings > 0]
  if positive_sims.empty:
    return user_ratings.mean()  # Fallback to mean rating if no similar items
  weighted_ratings = user_ratings[positive_sims.index] * positive_sims
  predicted_rating = weighted_ratings.sum() / positive_sims.sum()
  return predicted_rating if not np.isnan(predicted_rating) else user_ratings.mean()  
  

In [13]:
predicted_rating_alice = predict_ratings('Alice', 'Item5',example_df, item_similarity_matrix)
print(f"Predicted rating for Alice on Item5: {predicted_rating_alice:.2f}")

Predicted rating for Alice on Item5: 4.03


In [15]:
# Tải dữ liệu
columns = ['userId', 'itemId', 'rating', 'timestamp']
df = pd.read_csv('../data/ml-100k/u.data', sep='\t', names=columns)

# Chia dữ liệu thành 80% train và 20% test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Tạo ma trận User-Item từ tập train
train_user_item_matrix = train_df.pivot_table(index='userId', columns='itemId', values='rating')

In [16]:
# Chuyển vị và điền 0 vào các giá trị NaN
train_item_user_matrix = train_user_item_matrix.T.fillna(0)

# Tính toán ma trận tương đồng
print("Đang tính toán ma trận Item-Item Similarity...")
item_similarity_array = cosine_similarity(train_item_user_matrix)

# Chuyển thành DataFrame
train_item_similarity_matrix = pd.DataFrame(
    item_similarity_array,
    index=train_item_user_matrix.index,
    columns=train_item_user_matrix.index
)
print("Hoàn thành!")

print("\nMa trận Item-Item Similarity (5x5):")
display(train_item_similarity_matrix.iloc[:5, :5])

Đang tính toán ma trận Item-Item Similarity...
Hoàn thành!

Ma trận Item-Item Similarity (5x5):


itemId,1,2,3,4,5
itemId,,,,,
1,1.000000,0.311393,0.253690,0.350312,0.214229
2,0.311393,1.000000,0.216764,0.383544,0.304612
3,0.253690,0.216764,1.000000,0.261066,0.141570
4,0.350312,0.383544,0.261066,1.000000,0.270286
5,0.214229,0.304612,0.141570,0.270286,1.000000


In [18]:
predictions = []
true_ratings = []

# Lặp qua tập test để dự đoán
for _, row in test_df.iterrows():
    user_id = row['userId']
    item_id = row['itemId']
    true_rating = row['rating']
    
    # Chỉ dự đoán nếu user và item có trong tập train
    if user_id in train_user_item_matrix.index and item_id in train_item_similarity_matrix.index:
        # Sử dụng hàm đã xây dựng, truyền vào ma trận đã tính toán trước
        pred = predict_ratings(
            user_id, 
            item_id, 
            train_user_item_matrix, 
            train_item_similarity_matrix
        )
        
        predictions.append(pred)
        true_ratings.append(true_rating)

# Tính toán RMSE
rmse = np.sqrt(mean_squared_error(true_ratings, predictions))
print(f"Số lượng dự đoán thực hiện được: {len(predictions)}")
print(f"RMSE trên tập test: {rmse:.4f}")

Số lượng dự đoán thực hiện được: 19969
RMSE trên tập test: 1.0128
